In [1]:
import math
available_pieces =  [(21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 2), (28, 2), (29, 3), (30, 3), (31, 3), (32, 3), (33, 4), (34, 4), (35, 4), (36, 4)]

In [2]:
class Tree:
  def __init__(self):
    # How far in the tree are we ?
    self.previousValues = []
    # Parent 
    self.parent = None
    # Child number
    self.childNb = None
    # Probability to go into node
    self.probability = 1
    # Result from the leaf node
    self.result = 0
    # The six child possibilities for the dice or end turn 
    self.children = []

  def __str__(self, level=0):
    #if level == 5:
    #  return ""
    ret = "\t"*level+"child : "+repr(self.childNb)+", previous : "+repr(self.previousValues)+", result : "+repr(self.result)+", probability : "+repr(self.probability)+"\n"
    if self.children != []:
      for child in self.children:
        ret += child.__str__(level+1)
    return ret
  
def createNode(node, previousValues, childNb):
    newNode = Tree()
    newNode.previousValues.extend(node.previousValues)
    newNode.previousValues.extend(previousValues)
    newNode.parent = node
    newNode.childNb = childNb
    node.children.append(newNode)
    return newNode

In [3]:
def nonTerminal(node):
    if len(node.previousValues) == 8:
        return 0
    if len(list(set(node.previousValues))) == 6:
        return 0
    return 1

def CreateMarkovChain(node):
    if nonTerminal(node):
        for n in range(1,9):
            if n + len(node.previousValues) <= 8:
                for i in [1,2,3,4,5,"worm"]:
                    if i not in node.previousValues:
                        childNode = createNode(node, [i] * n, i)
                        CreateMarkovChain(childNode)
            else:
                break

def IsWinner(node, degree):
    if "worm" in node.previousValues:
        values = [i if i!="worm" else 5 for i in node.previousValues]
        if sum(values) > 20:
            if degree == 0:
                return 1
            elif degree == 1:
                values = [i if i!="worm" else 5 for i in node.parent.previousValues]
                if sum(values) > 20 and "worm" in node.parent.previousValues:
                    return 0
                else:
                    return 1
    return 0

def PruneNonWinningLeaves(node, degree):
    if node.children == []:
        if not IsWinner(node, degree):
            node.parent.children.remove(node)
            return 1
        else:
            return 0
    else:
        val = 0
        index = 0
        for i in node.children:
            val += PruneNonWinningLeaves(node.children[index], degree)
            index += 1
        return 1 if val > 0 else 0

def PruneNonWinningBranches(node, degree):
    while PruneNonWinningLeaves(node, degree):
        continue

def NodeProbabilityAndResult(node):
    probabilityDependingOnNbDice = [[1/6],[10/36, 1/36],[75 / 216 , 15 / 216 , 1 / 216],[500 / 1296 , 150 / 1296 , 20 / 1296 , 1 / 1296],[3125 / 7776 , 1250 / 7776 , 250 / 7776 , 25 / 7776 , 1 / 7776],[18750 / 46656 , 9375 / 46656 , 2500 / 46656 , 375 / 46656 , 30 / 46656 , 1 / 46656],[109375 / 279936 , 65625 / 279936 , 21875 / 279936 , 4375 / 279936 , 525 / 279936 , 35 / 279936 , 1 / 279936],[625000/1679616, 437500/1679616, 175000/1679616, 43750/1679616, 7000/1679616, 700/1679616, 40/1679616, 1/1679616]]
    probabilityPerNumber = [1/6, 11/36, 91/216, 671/1296, 4651/7776, 31031/46656, 201811/279936, 1288991/1679616]
    if node.children != []:
        probability = []
        for i in node.children:
            # probability.append(1/math.pow(6, i.previousValues.count(i.childNb)))
            probability.append(probabilityDependingOnNbDice[7-len(node.previousValues)][i.previousValues.count(i.childNb)-1])
        dice_left = 8-len(node.previousValues)-1
        sumProb = probabilityPerNumber[dice_left]*6
        # sumProb = sum(probability)
        probability = [i / sumProb for i in probability]
        for i in range(len(node.children)):
            node.children[i].probability = probability[i]
            NodeProbabilityAndResult(node.children[i])
    result(node)

# Result for node
def result(node):
    node.result = 0
    if "worm" in node.previousValues:
        values = [node.previousValues[j] if node.previousValues[j] != "worm" else 5 for j in range(len(node.previousValues))]
        score = sum(values)
        i = 0
        while score-i > 20:
            if score-i in [k[0] for k in available_pieces]:
                node.result = available_pieces[[y[0] for y in available_pieces].index(score-i)][1]
                break
            i += 1

In [4]:
# a = []
# for i in range(6):
#     for j in range(6):
#         for k in range(6):
#             if i == 1 or j == 1:
#                 a.append("1")
# a.count("1")

In [5]:
# a = []
# for b in range(6):
#     for c in range(6):
#         # for d in range(6):
#             # for e in range(6):
#                 # for f in range(6):
#                     # for g in range(6):
#                         # for h in range(6):
#                             # for i in range(6):
#                                 a.append([b,c])
#                                 # if b == 1 or c == 1 or d == 1 or e == 1 or f == 1 or g == 1 or h == 1 or i == 1:
#                                 #     # if (b == 1 and c == 1) or (b == 1 and d == 1) or (c == 1 and d == 1):
#                                 #         # if b == 1 and c == 1 and d == 1:
#                                 #         if b == 1 and c == 1 and d == 1 and e == 1 and f == 1 and g == 1 and h == 1 and i == 1:
#                                 #             a += 1
# # print(a)
# val = pow(6,2)
# print(sum([1 for i in a if i.count(0)==1]),"/",val,",", sum([1 for i in a if i.count(0)==2]),"/",val,",", sum([1 for i in a if i.count(0)==3]),"/",val,",", sum([1 for i in a if i.count(0)==4]),"/",val,",", sum([1 for i in a if i.count(0)==5]),"/",val,",", sum([1 for i in a if i.count(0)==6]),"/",val,",", sum([1 for i in a if i.count(0)==7]),"/",val,",", sum([1 for i in a if i.count(0)==8]),"/",val)
# sum([1 for i in a if 0 in i])
# print(pow(6,2))
# # print("occurence : " + str(a)+ ", throws : " +str(pow(6,8)))

In [6]:
# probabilityDependingOnNbDice = [[1/6],[10/36, 1/36],[75 / 216 , 15 / 216 , 1 / 216],[500 / 1296 , 150 / 1296 , 20 / 1296 , 1 / 1296],[3125 / 7776 , 1250 / 7776 , 250 / 7776 , 25 / 7776 , 1 / 7776],[18750 / 46656 , 9375 / 46656 , 2500 / 46656 , 375 / 46656 , 30 / 46656 , 1 / 46656],[109375 / 279936 , 65625 / 279936 , 21875 / 279936 , 4375 / 279936 , 525 / 279936 , 35 / 279936 , 1 / 279936],[625000/1679616, 437500/1679616, 175000/1679616, 43750/1679616, 7000/1679616, 700/1679616, 40/1679616, 1/1679616]]
# probabilityPerNumber = [1/6, 11/36, 91/216, 671/1296, 4651/7776, 31031/46656, 201811/279936, 1288991/1679616]
# print(probabilityDependingOnNbDice)

In [7]:
markovChain = Tree()
CreateMarkovChain(markovChain)
NodeProbabilityAndResult(markovChain)
# print(markovChain)
PruneNonWinningBranches(markovChain, 1)

In [8]:
markovChain = markovChain.children[8].children[2].children[0]
print(markovChain)

child : 1, previous : [3, 3, 4, 1], result : 0, probability : 0.11198308607467929
	child : 2, previous : [3, 3, 4, 1, 2], result : 0, probability : 0.1241927471435668
		child : 5, previous : [3, 3, 4, 1, 2, 5], result : 0, probability : 0.13736263736263737
			child : 'worm', previous : [3, 3, 4, 1, 2, 5, 'worm'], result : 1, probability : 0.15151515151515152
			child : 'worm', previous : [3, 3, 4, 1, 2, 5, 'worm', 'worm'], result : 2, probability : 0.01515151515151515
		child : 'worm', previous : [3, 3, 4, 1, 2, 'worm'], result : 0, probability : 0.13736263736263737
			child : 5, previous : [3, 3, 4, 1, 2, 'worm', 5], result : 1, probability : 0.15151515151515152
			child : 5, previous : [3, 3, 4, 1, 2, 'worm', 5, 5], result : 2, probability : 0.01515151515151515
		child : 5, previous : [3, 3, 4, 1, 2, 5, 5], result : 0, probability : 0.027472527472527476
			child : 'worm', previous : [3, 3, 4, 1, 2, 5, 5, 'worm'], result : 2, probability : 0.16666666666666666
		child : 'worm', previou

In [9]:
def checkNodes(node):
    # leaf case 
    if node.children == []:
        return node.probability * node.result
    # non leaf cases
    value = 0
    for i in range(len(node.children)):
        value += checkNodes(node.children[i])
    return value * node.probability

def bestProbabilities(node, rolls):
    roll = list( dict.fromkeys(rolls) )
    results_check = []
    for element in roll:
        if element not in node.previousValues:
            new_node = node 
            for i in range(len(new_node.children)):
                if rolls.count(element) == new_node.children[i].previousValues.count(element):
                    new_node = new_node.children[i]
                    results_check.append(checkNodes(new_node)/new_node.probability)
                    print(element)
                    break
    return results_check
rolls = ["worm", 5, 5, 2, 1]
print(bestProbabilities(markovChain, rolls))

worm
5
2
[0.2337662337662338, 0.398989898989899, 0.09024309024309025]


In [10]:
def treeBuilder(treeStr, node):
    if len(node.children) > 0:
        treeStr = treeStr + '('
        for child in node.children:
            treeStr = treeBuilder(treeStr, child)
            if not child == node.children[-1]:
                treeStr = treeStr + ','
            else:
                treeStr = treeStr + ')'
    for index, value in enumerate(node.previousValues):
        if index == 0:
            treeStr = treeStr + str(value)
        else:
            treeStr = treeStr + '_' + str(value)
    treeStr = treeStr + ':' + str(node.probability)
    
    return treeStr
            

In [11]:
visTree = treeBuilder('', markovChain)

In [12]:
visTree = visTree + ';'
print(visTree)

(((3_3_4_1_2_5_worm:0.15151515151515152,3_3_4_1_2_5_worm_worm:0.01515151515151515)3_3_4_1_2_5:0.13736263736263737,(3_3_4_1_2_worm_5:0.15151515151515152,3_3_4_1_2_worm_5_5:0.01515151515151515)3_3_4_1_2_worm:0.13736263736263737,(3_3_4_1_2_5_5_worm:0.16666666666666666)3_3_4_1_2_5_5:0.027472527472527476,3_3_4_1_2_worm_worm:0.027472527472527476,3_3_4_1_2_worm_worm_worm:0.0018315018315018315)3_3_4_1_2:0.1241927471435668,((3_3_4_1_5_2_worm:0.15151515151515152,3_3_4_1_5_2_worm_worm:0.01515151515151515)3_3_4_1_5_2:0.13736263736263737,3_3_4_1_5_worm:0.13736263736263737,(3_3_4_1_5_2_2_worm:0.16666666666666666)3_3_4_1_5_2_2:0.027472527472527476,3_3_4_1_5_worm_worm:0.027472527472527476,3_3_4_1_5_worm_worm_worm:0.0018315018315018315)3_3_4_1_5:0.1241927471435668,((3_3_4_1_worm_2_5:0.15151515151515152,3_3_4_1_worm_2_5_5:0.01515151515151515)3_3_4_1_worm_2:0.13736263736263737,3_3_4_1_worm_5:0.13736263736263737,(3_3_4_1_worm_2_2_5:0.16666666666666666)3_3_4_1_worm_2_2:0.027472527472527476,3_3_4_1_worm_5_5

In [13]:
visTree.count(')')

18

In [ ]:
import ete3 as ete

tree = ete.Tree(visTree, format=1)

ts = ete.TreeStyle()
lstyle = ete.NodeStyle()
lstyle["fgcolor"] = "blue"
lstyle["size"] = 1.5

nstyle = ete.NodeStyle()
nstyle["fgcolor"] = "red"
nstyle["size"] = 3

for n in tree.traverse():
    #print(n.name, n.is_leaf())
    if n.is_leaf():
        n.set_style(lstyle)
    else:
        n.add_face(ete.TextFace(n.name), column=0)
        n.set_style(nstyle)

#ts.mode = 'c'
        
tree.show(tree_style = ts)